In [1]:
# !pip install --quiet git+ssh://git.amazon.com/pkg/DigitalPolarsUtilities
# !pip install --force-reinstall --upgrade polars[pyarrow]
# !pip install --upgrade scikit-learn
# !pip install torcheval
# !pip install tokenizers
# !pip install torch

import json
import os
import random
import uuid
import torch
from importlib import reload

from secure_ai_sandbox_python_lib.session import Session
from tokenizers import Tokenizer

MODEL_PATH = "models_current/model.pt"
TOKENIZER_PATH = "tokenizer.json"

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[04/15/25 21:36:20] INFO     Found credentials from IAM Role:                                   ]8;id=496691;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=555234;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


                    INFO     CA certs are provided via the AmazonCACerts installation at     ]8;id=296197;file:///home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts/cacerts_helpers.py\cacerts_helpers.py]8;;\:]8;id=463123;file:///home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts/cacerts_helpers.py#116\116]8;;\
                             /home/ec2-user/.local/lib/python3.10/site-packages/amazoncerts                        

Unable to load rust_cloudauth_core Module


In [2]:
!mkdir upload
!mkdir upload/code
!cp {MODEL_PATH} upload/model.pt
!cp {TOKENIZER_PATH} upload/tokenizer.json
!cp lstm2risk.py upload/code/
!cp requirements.txt upload/code/

In [49]:
%%writefile {os.path.join("upload", "code", "inference.py")}
import os
import json
import logging
import unicodedata

import torch
import lstm2risk

from tokenizers import Tokenizer

torch.set_num_threads(1)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

TEXT_FEATURE_NAMES = [
    'emailAddress',
    'billingAddressName',
    'customerName',
    'paymentAccountHolderName'
]

TABULAR_FEATURE_NAMES = [
    "billingAddressAge",
    "daysSinceFirstCompletedOrderForFingerprint",
    "deviceIdAge",
    "evMaxLinkScoreOfAllCategory",
    "evMeanLinkScoreOfAllCategory",
    "fingerprintAge",
    "geoBillCountryCcCountryCodeEqual",
    "ip2Age",
    "ip3Age",
    "ipAge",
    "numNewSigninCountries21Days",
    "numNewSigninTimezones21Days",
    "numNewSigninTimezones7Days",
    "osNameAge",
    "paymentInstrumentAgeInDays",
    "pcDigitalOrderTotalForCustomerIdIn1HourUSD",
    "pcDigitalOrderTotalForCustomerIdIn20MinsUSD",
    "pcDigitalOrderTotalForIpIn1DayUSD",
    "pcDigitalOrderTotalForIpIn1HourUSD",
    "pcDigitalOrderTotalForIpIn7DaysUSD",
    "pcDigitalOrderTotalForUbidIn1HourUSD",
    "pcItemSubtotalSameASINForFingerprintIn15DaysUSD",
    "pcItemSubtotalSameASINForFingerprintIn1DayUSD",
    "pcItemSubtotalSameASINForFingerprintIn1HourUSD",
    "pcItemSubtotalSameGLForCustomerIdIn1HourUSD",
    "pcItemSubtotalSameGLForIpIn15DaysUSD",
    "pcItemSubtotalSameGLForIpIn1DayUSD",
    "pcNumDigitalOrdersForPaymentTokenIn1Day",
    "pcNumDigitalOrdersForPaymentTokenIn1Hour",
    "pcNumDigitalOrdersForPaymentTokenIn7Days",
    "pcNumGiftCardPaidDigitalOrdersForIpIn15Days",
    "pcNumGiftCardPaidDigitalOrdersForIpIn1Day",
    "pcNumGiftCardPaidDigitalOrdersForIpIn1Hour",
    "pcNumGiftCardPaidDigitalOrdersForUbidIn1Day",
    "pcNumGiftCardPaidDigitalOrdersForUbidIn1Hour",
    "pcNumItemsSameASINForPaymentTokenIn15Days",
    "pcNumItemsSameASINForPaymentTokenIn1Day",
    "pcNumItemsSameASINForPaymentTokenIn1Hour",
    "pcNumItemsSameGLForPaymentTokenIn15Days",
    "pcNumItemsSameGLForPaymentTokenIn1Day",
    "pcNumItemsSameGLForPaymentTokenIn1Hour",
    "pcNumPaymentTokensUsedInLast1Hour",
    "pcNumPaymentTokensUsedInLast7Days",
    "pcOrderTotalForCustomerIdIn1HourUSD",
    "pcOrderTotalForIpIn1DayUSD",
    "pcOrderTotalForIpIn1HourUSD",
    "pcOrderTotalForPaymentTokenIn1HourUSD",
    "pcOrderTotalForUbidIn1HourUSD",
    "timeZoneChanged",
    "ubidAge",
]

MISSING_TOKEN = "[MISSING]"
SEPARATOR_TOKEN = "|"


def model_fn(model_dir: str) -> dict[str, lstm2risk.LSTM2Risk | Tokenizer]:
    """
    Load the model and tokenizer from the specified directory.

    Args:
        model_dir: Directory containing model artifacts

    Returns:
        Dicttionary of model, tokenizer
    """
    logger.info(f"Loading model from {model_dir}")

    try:
        tokenizer = Tokenizer.from_file(os.path.join(model_dir, "tokenizer.json"))
        model = lstm2risk.LSTM2Risk().to(DEVICE)
        model.load_state_dict(
            torch.load(os.path.join(model_dir, "model.pt"), map_location=DEVICE)
        )
        model.eval()
        assert lstm2risk.N_EMBED == tokenizer.get_vocab_size()
        return {"model": model, "tokenizer": tokenizer}
    except Exception as e:
        logger.error(f"Error loading model: {e}")
        raise


def input_fn(request_body: str, content_type: str) -> dict[str, str | list[float]]:
    """
    Parse and validate input data.

    Args:
        request_body: JSON string containing input data
        content_type: Expected content type of input

    Returns:
        Dictionary containing processed text and tabular inputs
    """
    assert content_type == "application/json"
    try:
        input_dict = json.loads(request_body)
        
        # Handles completely missing keys with the -1 default for tabular features
        tabular_inputs = [float(input_dict.get(feature, -1)) for feature in TABULAR_FEATURE_NAMES]

        # Handles missing keys, empty strings, and None values for text features
        text_inputs = [str(input_dict.get(feature) or MISSING_TOKEN) for feature in TEXT_FEATURE_NAMES]
        
        return {
            "text_input": SEPARATOR_TOKEN.join(text_inputs),
            "tabular_input": tabular_inputs,
        }
    except Exception as e:
        logger.error(f"Input processing error: {e} for request {request_body}")
        raise


@torch.inference_mode()
def predict_fn(data: dict[str, str | list[float]], model_dict: dict[str, lstm2risk.LSTM2Risk | Tokenizer]) -> float:
    """
    Generate prediction from processed input data.

    Args:
        data: Processed input data
        model_and_tokenizer: Dict of model, tokenizer

    Returns:
        Prediction score
    """
    try:
        model, tokenizer = model_dict["model"], model_dict["tokenizer"]
        text_input, tabular_input = data["text_input"], data["tabular_input"]
        tokenized_text_input = tokenizer.encode(unicodedata.normalize('NFKC', text_input)).ids
        model_input = {
            "text": torch.tensor([tokenized_text_input], device=DEVICE, dtype=torch.int),
            "tabular": torch.tensor([tabular_input], device=DEVICE, dtype=torch.float)
        }
        return model(model_input).item()
    except Exception as e:
        logger.error(f"Prediction error: {e} on input {data}")
        raise


def output_fn(prediction: float, accept: str) -> str:
    """
    Format the prediction output.

    Args:
        prediction: Model prediction
        accept: Expected output format

    Returns:
        JSON string containing formatted prediction
    """
    assert accept == "application/json"
    return json.dumps({
        "predictions": [{
            "legacy-score": prediction,
            "calibrated-score": prediction
        }]
    })

Overwriting upload/code/inference.py


In [50]:
def import_inference():
    import importlib
    import os
    python_file = "upload/code/inference.py"
    file_path = os.path.abspath(python_file)

    # Load the module dynamically
    spec = importlib.util.spec_from_file_location("inference", file_path)
    inference = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(inference)

    # Import the functions
    for x in ["model_fn", "input_fn", "predict_fn", "output_fn"]:
        assert x in inference.__dict__.keys(), f"Missing {x} in {python_file}"
    model_fn = inference.model_fn
    input_fn = inference.input_fn
    predict_fn = inference.predict_fn
    output_fn = inference.output_fn
    return model_fn, input_fn, predict_fn, output_fn

model_fn, input_fn, predict_fn, output_fn = import_inference()

In [51]:
with open("features.txt") as file:
    tabular_features = {line.strip(): "NUMERIC" for line in file}

In [52]:
import random
from tqdm.notebook import tqdm

In [54]:
%%time

model_and_tokenizer = model_fn("models_current")
features = {"emailAddress", "billingAddressName", "customerName", "paymentAccountHolderName"} | {feature for feature in tabular_features}
print(f"Number of Parameters: {sum(p.numel() for p in model_and_tokenizer['model'].parameters())}")

for row in tqdm(range(10_000)):
    input_data = json.dumps({'emailAddress':20 * 'a', 'billingAddressName': 20 * '1', 'customerName':20 * 'C', 'paymentAccountHolderName':20 * 'd'} | {k: random.random() for k in tabular_features})
    # input_data = {k: v for k, v in row.items() if k in features}
    input_dict = input_fn(input_data, "application/json")
    prediction = predict_fn(input_dict, model_and_tokenizer)
    output_fn(prediction, "application/json")

In [25]:
import uuid
from secure_ai_sandbox_python_lib.session import Session

model_id = uuid.uuid4()

s3_model_path = f"s3://{os.path.join(Session(session_folder='./').my_owned_s3_bucket_name(), f'names2risk/{model_id}.tar.gz')}"
s3_model_path

[04/15/25 21:45:15] INFO     Found credentials from IAM Role:                                   ]8;id=23919;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=531774;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/15/25 21:45:16] INFO     successfully patched module botocore                                    ]8;id=740596;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/patcher.py\patcher.py]8;;\:]8;id=277959;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/patcher.py#119\119]8;;\

's3://sandboxuserdependency-kkrowpma-personals3bucket-ytedbyl0mqm/names2risk/54a94470-25c1-46f5-91ce-e9dea241847e.tar.gz'

In [26]:
!cd upload && tar -czf ../{model_id}.tar.gz ./*
!aws s3 cp {model_id}.tar.gz {s3_model_path}

upload: ./54a94470-25c1-46f5-91ce-e9dea241847e.tar.gz to s3://sandboxuserdependency-kkrowpma-personals3bucket-ytedbyl0mqm/names2risk/54a94470-25c1-46f5-91ce-e9dea241847e.tar.gz


In [27]:
import sagemaker
from sagemaker.pytorch import PyTorchModel

model = PyTorchModel(
    role=sagemaker.get_execution_role(),
    model_data=s3_model_path,
    framework_version="2.2",
    py_version="py310",
)

[04/15/25 21:45:40] INFO     Found credentials from IAM Role:                                   ]8;id=16411;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=903540;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [28]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

                    INFO     Found credentials from IAM Role:                                   ]8;id=774106;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=425352;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/15/25 21:45:41] INFO     Creating model with name: pytorch-inference-2025-04-15-21-45-41-507    ]8;id=833343;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=184654;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Add currentOwnerAlias tag to the request for operation:             ]8;id=916625;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=929709;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#92\92]8;;\
                             CreateModel.                                                                          

                    INFO     A creation operation CreateModel is detected. Apply owner tag to   ]8;id=592814;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=584714;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#111\111]8;;\
                             the request.                                                                          

                    WARNING  SAIS Patcher is going to apply security configuration into request  ]8;id=172202;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/utils/patch_helper.py\patch_helper.py]8;;\:]8;id=97017;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/utils/patch_helper.py#36\36]8;;\
                             content for operation: CreateModel, with value for field:                             
                             VpcConfig,Subnets to Secure AI Sandbox provided configuration                         
                             value: ['subnet-018ad8f365487e2c3', 'subnet-07c19f31d601b8416',                       
                             'subnet-0f8bb2837ed1bd5c3', 'subnet-09c7860b6c3e61a97',                               
                             'subnet-03838652b3810f147'].                                                          

...

                    WARNING  SAIS Patcher is going to apply security configuration into request  ]8;id=106507;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/utils/patch_helper.py\patch_helper.py]8;;\:]8;id=859804;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/utils/patch_helper.py#36\36]8;;\
                             content for operation: CreateEndpointConfig, with value for field:                    
                             KmsKeyId to Secure AI Sandbox provided configuration value:                           
                             arn:aws:kms:us-east-1:264840978154:key/8eb72029-4832-492f-97b1-1fe1                   
                             b4d6e6cb.                                                                             

                    INFO     Creating endpoint with name pytorch-inference-2025-04-15-21-45-42-324  ]8;id=964582;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=985114;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

                    INFO     Extended currentOwnerAlias tag to the request for operation:        ]8;id=511639;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=770076;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#97\97]8;;\
                             CreateEndpoint.                                                                       

                    INFO     A creation operation CreateEndpoint is detected. Apply owner tag   ]8;id=915536;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=708398;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#111\111]8;;\
                             to the request.                                                                       

------!

In [29]:
predictor.predict({'emailAddress':20*'a', 'billingAddressName': 20*'1', 'customerName': 20*'C', 'paymentAccountHolderName': 20*'d'} | {k: random.random() for k in tabular_features})

{'predictions': [{'legacy-score': 0.5294486880302429,
   'calibrated-score': 0.5294486880302429}]}

In [30]:
import boto3

def get_initialization_errors(endpoint_name):
    """Get initialization errors for a SageMaker endpoint"""
    client = boto3.client('logs')
    log_group = f"/aws/sagemaker/Endpoints/{endpoint_name}"
    
    try:
        # Get log streams
        response = client.describe_log_streams(
            logGroupName=log_group,
            orderBy='LastEventTime',
            descending=True
        )

        for stream in response['logStreams']:
            logs = client.get_log_events(
                logGroupName=log_group,
                logStreamName=stream['logStreamName']
            )

            # Look for initialization messages
            for event in logs['events']:
                if any(error_term in event['message'].lower() 
                      for error_term in ['error', 'exception', 'failed']):
                    print(f"Found error: {event['message']}")

    except client.exceptions.ResourceNotFoundException:
        print(f"No logs found for endpoint {endpoint_name}")

# Check for initialization errors
get_initialization_errors(predictor.endpoint_name)

[04/15/25 21:49:14] INFO     Found credentials from IAM Role:                                   ]8;id=262581;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=463792;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [31]:
import time
from concurrent.futures import ThreadPoolExecutor
import numpy as np

def time_prediction(predictor, payload):
    start = time.time()
    response = predictor.predict(payload)
    end = time.time()
    return end - start

def run_prediction():
    input_data = {'emailAddress':20*'a', 'billingAddressName': 20*'1', 'customerName': 20*'C', 'paymentAccountHolderName': 20*'d'} | {k: random.random() for k in tabular_features}
    return time_prediction(predictor, input_data)

n_requests = 10_000
with ThreadPoolExecutor(max_workers=10) as executor:
    latencies = list(executor.map(lambda _: run_prediction(), range(n_requests)))

print(f"Average latency: {np.mean(latencies):.3f}s")
print(f"P50 latency: {np.percentile(latencies, 50):.3f}s")
print(f"P95 latency: {np.percentile(latencies, 95):.3f}s")
print(f"P99 latency: {np.percentile(latencies, 99):.3f}s")

Average latency: 0.097s
P50 latency: 0.091s
P95 latency: 0.138s
P99 latency: 0.179s


In [32]:
predictor.delete_model()
predictor.delete_endpoint()

[04/15/25 21:51:16] INFO     Deleting model with name: pytorch-inference-2025-04-15-21-45-41-507    ]8;id=848689;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=335024;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#5356\5356]8;;\

                    WARNING  A modification operation DeleteModel is detected. Please make sure ]8;id=804737;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=53764;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#118\118]8;;\
                             the resource to be modified is owned or initially created by you;                     
                             Otherwise your request will be denied by target service due to                        
                             lack of permission.                                                                   

                    INFO     Deleting endpoint configuration with name:                             ]8;id=402769;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=905175;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4995\4995]8;;\
                             pytorch-inference-2025-04-15-21-45-42-324                                             

                    WARNING  A modification operation DeleteEndpointConfig is detected. Please  ]8;id=764257;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=755113;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#118\118]8;;\
                             make sure the resource to be modified is owned or initially                           
                             created by you; Otherwise your request will be denied by target                       
                             service due to lack of permission.                                                    

[04/15/25 21:51:17] INFO     Deleting endpoint with name: pytorch-inference-2025-04-15-21-45-42-324 ]8;id=408161;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=895079;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4985\4985]8;;\

                    WARNING  A modification operation DeleteEndpoint is detected. Please make   ]8;id=661620;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py\secure_patch.py]8;;\:]8;id=225128;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_patcher/secure_config_patcher/ext/botocore/secure_patch.py#118\118]8;;\
                             sure the resource to be modified is owned or initially created by                     
                             you; Otherwise your request will be denied by target service due                      
                             to lack of permission.                                                                

In [33]:
with open("features.txt") as file:
    tabular_features = {line.strip(): "NUMERIC" for line in file}

features = tabular_features | {"emailAddress": "TEXT", "billingAddressName": "TEXT", "customerName": "TEXT", "paymentAccountHolderName": "TEXT"}

In [34]:
sample_payload = json.dumps({"emailAddress": "a" * 20, "billingAddressName": "b" * 20, "customerName": "c" * 20, "paymentAccountHolderName": "d" * 20} | {feature: random.random() for feature in tabular_features})
bucket_name = Session(session_folder='./').my_owned_s3_bucket_name()
with open("/tmp/sample_payload.json", "w") as file:
    file.write(sample_payload)
!cd /tmp/ && tar -cvpzf sample_payload.json.tar.gz sample_payload.json
!aws s3 cp /tmp/sample_payload.json.tar.gz s3://{bucket_name}/names2risk/sample_payload.json.tar.gz

[04/15/25 21:51:21] INFO     Found credentials from IAM Role:                                   ]8;id=867748;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=325521;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sample_payload.json
upload: ../../../../tmp/sample_payload.json.tar.gz to s3://sandboxuserdependency-kkrowpma-personals3bucket-ytedbyl0mqm/names2risk/sample_payload.json.tar.gz


In [35]:
f"s3://{bucket_name}/names2risk/sample_payload.json.tar.gz"

's3://sandboxuserdependency-kkrowpma-personals3bucket-ytedbyl0mqm/names2risk/sample_payload.json.tar.gz'

In [36]:
!aws s3 ls s3://sandboxuserdependency-kkrowpma-personals3bucket-ytedbyl0mqm/names2risk/

2025-04-01 00:12:22   18061116 07d4d91a-056a-4877-abb6-7dceb03ee400.tar.gz
2025-04-09 22:23:31   18061155 18ec160a-3437-4ad5-a8d6-3a054f5859d8.tar.gz
2025-03-28 20:49:23   18061124 234db9d3-e9ce-4009-913a-0eadfdfdc2eb.tar.gz
2025-04-01 03:44:28   18061116 41854fa8-deb6-4773-9b25-a00e95758fd0.tar.gz
2025-04-01 20:38:18   18061133 4f6e7045-8bbf-4957-8d8e-776b40fc616e.tar.gz
2025-03-31 19:52:44   18061096 541c4692-3f03-4f7b-80f4-02c6742ced9e.tar.gz
2025-04-15 21:45:26   18058886 54a94470-25c1-46f5-91ce-e9dea241847e.tar.gz
2025-04-07 23:47:50   18061197 8837e7da-1146-4d0a-979c-42c71936865b.tar.gz
2025-04-07 23:39:50   18061162 d46c1057-4dbf-43eb-b7bc-93d628c553ee.tar.gz
2025-04-09 22:32:11   18061289 df693ffa-41f2-481d-a4a3-a0f305a62119.tar.gz
2025-04-09 22:31:15   18061265 ff840a77-c6dd-4fe5-9956-a4ee1df00b49.tar.gz
2025-04-15 21:51:23       1204 sample_payload.json.tar.gz


In [37]:
model_s3_key = os.path.basename(s3_model_path)
s3_bucket_name = os.path.dirname(s3_model_path)
s3_bucket_name, model_s3_key

('s3://sandboxuserdependency-kkrowpma-personals3bucket-ytedbyl0mqm/names2risk',
 '54a94470-25c1-46f5-91ce-e9dea241847e.tar.gz')

In [41]:
model_s3_key

'54a94470-25c1-46f5-91ce-e9dea241847e.tar.gz'

In [42]:
import sagemaker

model_registration_region = "NA"
model_objective = "DigitalEmail2RiskModelNA"
model_domain = "FORTRESS"
deploy_region = "us-east-1"
output_var_dict = {
    "legacy-score": "NUMERIC",
    "calibrated-score": "NUMERIC"
}
instance_type = "ml.m5.xlarge"

deploy_container_image = sagemaker.image_uris.retrieve(
    framework="pytorch",
    version="2.2",
    py_version="py310",
    region=deploy_region,
    image_scope="inference",
    instance_type=instance_type,
)

# Create the MIMS resource
mims = Session(session_folder='./').resource("MIMSModelRegistrar")

# Call register_model with your inputs
sagemaker_model = mims.register_model(
    model_owner="amzn1.abacus.team.xkuchlojq7lt4nir5uma",  # https://permissions.amazon.com/a/team/BRP-ML-Payment-Risk
    # The domain and objective to register your model in (this is where you will find your model on DAWS)
    model_domain=model_domain,
    model_objective=model_objective,
    # Currently support application/json or text/csv
    source_model_inference_content_types=["application/json"],
    source_model_inference_response_types=["application/json"],
    # Provide a dictionary mapping the variable name to the variable type for both input and output vars
    # The variable types supported are 'TEXT' and 'NUMERIC'
    source_model_inference_input_variable_list=features,
    source_model_inference_output_variable_list=output_var_dict,
    # The region that your SageMaker model will be registered in, should be NA/EU/FE
    model_registration_region=model_registration_region,
    # The image arn for your trained model to be registered
    source_model_inference_image_arn=deploy_container_image,
    # The region your model was trained in and the model artifacts are located in
    # (optional) If not provided, this value will be set to 'us-east-1'
    source_model_region=deploy_region,
    # The name of the S3 bucket your model artifacts are located in
    # (optional) If not provided, this value will be set to your personal sandbox S3 bucket- sandbox_session.my_owned_s3_bucket_name()
    source_model_artifact_s3_bucket=Session(session_folder='./').my_owned_s3_bucket_name(),
    # The S3 key to model artifact location inside your S3 bucket, should end in a .tar file
    source_model_artifact_s3_key=f"names2risk/{model_s3_key}",
    # (optional) You can provide a dictionary of environment variables to be associated with your model
    source_model_environment_variable_map={},
    load_testing_info_map={
        "sample_payload_s3_bucket": Session(session_folder='./').my_owned_s3_bucket_name(),
        "sample_payload_s3_key": "names2risk/sample_payload.json.tar.gz", 
        "max_acceptable_error_rate": 0.01,
        "expected_tps": 500, 
        "max_latency_in_millisecond": 100,
        "instance_type_list": [instance_type],
    },
)

# You can get the model id and workflow execution arn for your newly registered model
print("model id = ", sagemaker_model.get_model_id())
print("workflow_execution_arn =", sagemaker_model.get_workflow_execution_arn())

[04/16/25 03:29:39] INFO     Found credentials from IAM Role:                                   ]8;id=385794;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=234089;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/16/25 03:29:40] INFO     Found credentials from IAM Role:                                   ]8;id=897971;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=394617;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/16/25 03:29:43] INFO     Found credentials from IAM Role:                                   ]8;id=99835;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=883575;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/16/25 03:29:46] INFO     Found credentials from IAM Role:                                   ]8;id=704377;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=256282;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/16/25 03:29:51] INFO     Successfully PutMetricData: Invocation.                        ]8;id=368919;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_lib/cloudwatch/cloudwatch_client.py\cloudwatch_client.py]8;;\:]8;id=150178;file:///home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_lib/cloudwatch/cloudwatch_client.py#60\60]8;;\

model id =  2025-04-16-12587-short-force
workflow_execution_arn = arn:aws:states:us-east-1:843565719569:execution:MIMSRegisterModelStateMachine-Prod-NA:fa6b44ad-62e3-4c48-b27d-76b585338778


In [45]:
from time import sleep
import json
from IPython.display import clear_output, display
for i in range(100):
    status = sagemaker_model.get_workflow_status().execution_status
    if status != 'SUCCEEDED':
        display(status)
        clear_output(wait=True)
        sleep(30)
        continue
    else:
        display(status)
        # When workflow stops, you can print the output as a Json String
        display(json.loads(sagemaker_model.get_workflow_status().execution_output))
        with open("output.json", "w") as file:
            file.write(sagemaker_model.get_workflow_status().execution_output)
        break

Exception ignored in: <finalize object at 0x7fadac949d40; dead>
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/weakref.py", line 586, in __call__
    def __call__(self, _=None):
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fae961c9cc0>>
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   if status != 'SUCCEEDED':                                                               │
│    7 │   │   display(status)                                                                     │
│    8 │   │   clear_output(wait=True)                                                             │
│ ❱  9 │   │   sleep(30)                                                                           │
│   10 │   │   continue                                                                            │
│   11 │   else:                                                                                   │
│   12 │   │   display(status)                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [83]:
from secure_ai_sandbox_python_lib.session import Session

# Initialize sandbox_session
sandbox_session = Session(session_folder='/tmp/temp_folder', retail_region=model_registration_region)
mims = sandbox_session.resource('MIMSModelRegistrar')

endpoint_scaling_config_map = {
            "instance_type": "ml.m5.xlarge",
            "min_capacity": 6,
            "max_capacity": 12,
            "scaling_policy_map": {
                "version": "1.0",
                "target_value": 2919.5, # Maximum traffic per minutes one instance can support, 6000TPM = 100TPS
                "scale_in_cooldown": 300, # seconds wait for delete instance
                "scale_out_cooldown": 60, # seconds wait for add instance
                "disable_scale_in": False
            }
        }

response = mims.set_scaling_policy(
    model_region=model_registration_region,
    model_domain=model_domain,
    model_objective=model_objective,
    model_id=sagemaker_model.get_model_id(),
    endpoint_scaling_config_map=endpoint_scaling_config_map
)

                    INFO     Found credentials from IAM Role:                                   ]8;id=953024;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=515124;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[04/11/25 03:05:17] INFO     Found credentials from IAM Role:                                   ]8;id=560770;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=806832;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [50]:
sagemaker_model.get_model_id()

'2025-03-28-80770-tough-paste'

In [46]:
from secure_ai_sandbox_python_lib.session import Session

# Initialize sandbox_session
sandbox_session = Session(session_folder='/tmp/temp_folder', retail_region=model_registration_region)

# Create the MIMS resource
mims = sandbox_session.resource('MIMSModelRegistrar')

invoke_response = mims.invoke_sagemaker_endpoint(
    model_region=model_registration_region, 
    model_domain=model_domain,
    model_objective=model_objective,
    model_id=sagemaker_model.get_model_id(),
    payload=sample_payload
)
invoke_response

[03/28/25 21:35:42] INFO     Found credentials from IAM Role:                                   ]8;id=660149;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=190658;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/28/25 21:35:43] INFO     Found credentials from IAM Role:                                   ]8;id=786819;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=483548;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 # Create the MIMS resource                                                                  │
│    7 mims = sandbox_session.resource('MIMSModelRegistrar')                                       │
│    8                                                                                             │
│ ❱  9 invoke_response = mims.invoke_sagemaker_endpoint(                                           │
│   10 │   model_region=model_registration_region,                                                 │
│   11 │   model_domain=model_domain,                                                              │
│   12 │   model_objective=model_objective,                                                        │
│                                                                                                  │
│ /home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_lib/mims_model_regis │
│ tration/model_inference_management_service.py:139 in invoke_sagemaker_endpoint                   │
│                                                                                                  │
│   136 │   │   :return: InvokeSageMakerEndpointResponse                                           │
│   137 │   │   """                                                                                │
│   138 │   │                                                                                      │
│ ❱ 139 │   │   return self.invoke_sagemaker_endpoint_wrapper.invoke_sagemaker_endpoint(model_re   │
│   140 │   │   │   │   │   │   model_domain, model_objective, model_id, payload)                  │
│   141 │                                                                                          │
│   142 │   def set_alarm_threshold(self, model_region: str, alarm_type: AlarmType, model_id: st   │
│                                                                                                  │
│ /home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_lib/mims_model_regis │
│ tration/invoke_sagemaker_endpoint_wrapper.py:43 in invoke_sagemaker_endpoint                     │
│                                                                                                  │
│   40 │   │   │   requester=self._owner_alias                                                     │
│   41 │   │   )                                                                                   │
│   42 │   │                                                                                       │
│ ❱ 43 │   │   response_body = self._call_mims_api(request, model_region)                          │
│   44 │   │   coral_response = InvokeSageMakerEndpointResponse.__from_coral__(**response_body)    │
│   45 │   │   return coral_response                                                               │
│   46                                                                                             │
│                                                                                                  │
│ /home/ec2-user/.local/lib/python3.10/site-packages/secure_ai_sandbox_python_lib/mims_model_regis │
│ tration/base_mims_api_wrapper.py:61 in _call_mims_api                                            │
│                                                                                                  │
│    58 │   │   session = self._create_http_session(REGIONS_SYMBOL_TO_NAME_MAP[region])            │
│    59 │   │   resp_pay_load = session.post(f'{endpoint}/{self._api_uri}',                        │
│    60 │   │   │   │   │   │   │   │   │    data=json.dumps(coral_utils.convert_coral_to_dict(r   │
│ ❱  61 │   │   response_body = self._parse_response_to_dict(

In [67]:
# import boto3
# import time
# from datetime import datetime, timedelta


# def get_initialization_errors(endpoint_name):
#     """Get initialization errors for a SageMaker endpoint"""
#     client = boto3.client('logs')
#     log_group = f"/aws/sagemaker/Endpoints/{endpoint_name}"

#     try:
#         # Get log streams
#         response = client.describe_log_streams(
#             logGroupName=log_group,
#             orderBy='LastEventTime',
#             descending=True
#         )

#         for stream in response['logStreams']:
#             logs = client.get_log_events(
#                 logGroupName=log_group,
#                 logStreamName=stream['logStreamName']
#             )

#             # Look for initialization messages
#             for event in logs['events']:
#                 if any(error_term in event['message'].lower() 
#                       for error_term in ['error', 'exception', 'failed']):
#                     print(f"{event['message']}")

#     except client.exceptions.ResourceNotFoundException:
#         print(f"No logs found for endpoint {endpoint_name}")

# # Check for initialization errors
# get_initialization_errors(predictor.endpoint_name)